In [1]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [2]:
# Cargar tu AnnData
adata = sc.read('/home/mdiaz/scRNA-seq_LungProject/atlas_reference/data/processed/anotado_modelo_nativo_celltypist.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 78317 × 42771
    obs: 'suspension_type', 'donor_id', 'is_primary_data', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'tissue_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'cause_of_death', 'core_or_extension', 'dataset', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'smoking_status', 'study', 'subject_type', 'tissue_coarse_unharmonized', 't

In [4]:
from scipy import sparse
from scipy.io import mmwrite
import pandas as pd

# Extrae matriz de conteos
counts = adata.raw.X if adata.raw is not None else adata.X

# Transponer si está en formato sparse
if sparse.issparse(counts):
    mmwrite("counts.mtx", counts.T)  # ← IMPORTANTE: transpuesta
else:
    pd.DataFrame(counts.T.toarray(), index=adata.var_names, columns=adata.obs_names).to_csv("counts.csv")

# Guardar nombres
# genes = filas en R
# Para los nombres de genes correspondientes a adata.raw.X
gene_names = adata.raw.var_names if adata.raw is not None else adata.var_names
gene_names.to_series().to_csv("genes.tsv", sep="\t", index=False, header=False)

# células = columnas en R
adata.obs_names.to_series().to_csv("barcodes.tsv", sep="\t", index=False, header=False)

